<center>
<img src="https://drive.google.com/uc?id=1f1gGVI-rxcHjA90WEGNvvtSXF1pAxQwg" alt="Fasilkom UI" width="300"/>

CSCE604137 • Penambangan Data

Semester Ganjil 2021/2022

Fakultas Ilmu Komputer, Universitas Indonesia

##**Tugas Kelompok 1**

###**Tenggat Waktu: 15 April 2022, 22.00 WIB**
</center>

####**Ketentuan:**

Anggota Kelompok:
- Amanda Carrisa Ashardian - 1906696969
- Muhamad Yoga Mahendra - 1806205256
- Nabila Fathia Zahra - 1706022741


In [5]:
# Import
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import scipy.stats as scp
import seaborn as sn
import sklearn as sk
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
!pip install PySastrawi
!pip install scikit-learn
!pip install xgboost

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Converting csv to dataframe
contoh = pd.read_csv('/content/drive/MyDrive/Data TK 1 Text Mining/clean/contoh_data.csv')
human_ant = pd.read_csv('/content/drive/MyDrive/Data TK 1 Text Mining/clean/human_annotated.csv')
machine_ant = pd.read_csv('/content/drive/MyDrive/Data TK 1 Text Mining/clean/machine_annotated.csv')
testing = pd.read_csv('/content/drive/MyDrive/Data TK 1 Text Mining/clean/testing_data.csv')
unlabeled = pd.read_csv('/content/drive/MyDrive/Data TK 1 Text Mining/clean/unlabeled_data.csv')

In [7]:
# Normalizing text & removing stopwords
import re

def normalize(text):
    res = text.lower().strip() # Mengubah uppercase menjadi lowercase dan melakukan trimming pada teks
    res = re.sub("\d", "", res) # Menghilangkan angka
    res = re.sub("\s+", " ", res) # Menghilangkan spasi berlebih
    res = re.sub("[^\w\s]", "", res) # Menghilangkan tanda baca
    return res

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

stop_factory = StopWordRemoverFactory()
lst_stopword = stop_factory.get_stop_words()

# Fungsi untuk menghilangkan stopword dari suatu teks

def remove_stopword(text):
    lst_token = text.split(" ")
    res = []
    for token in lst_token:
        if token not in lst_stopword:
            res.append(token)
    return " ".join(res)

def clean_words(arr):
  corpus = []
  for text in arr:
    normalized_text = normalize(text)
    cleaned_text = remove_stopword(normalized_text)
    corpus.append(cleaned_text)
  return corpus
def replace_with_cleaned(arr,arr_clean):
  for i in range(len(arr_clean)):
    arr[i] = arr_clean[i]

In [8]:
training_data = machine_ant.append(human_ant, ignore_index = True)
training_data = training_data.append(contoh, ignore_index = True)

In [9]:
training_isi = training_data['isi'].astype(str)
training_clean = clean_words(training_isi)

testing_isi = testing['isi'].astype(str)
testing_clean = clean_words(testing_isi)

replace_with_cleaned(training_data['isi'],training_clean)
replace_with_cleaned(testing['isi'],testing_clean)

training_data['isi'] = training_data['judul']+training_data['isi']
testing['isi'] = testing['judul']+testing['isi']
testing_data = testing.copy()
testing_data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,judul,isi
0,DS-45,kenapa telinga berdengung tidak bisa hilang ?,kenapa telinga berdengung tidak bisa hilang ?s...
1,DS-339,apabila anak anda lambat berbicara,apabila anak anda lambat berbicarakenal team d...
2,AD-18,sakit perut setiap habis makan dan minum,sakit perut setiap habis makan dan minumselama...
3,AD-66,cara mengobati vertigo berkepanjangan,cara mengobati vertigo berkepanjanganselamat m...
4,AD-73,imunisasi dapat dan polio,imunisasi dapat dan poliomalam dok anak umur ...
...,...,...,...
2922,KD-43404,keluhan setelah berhubungan,keluhan setelah berhubungandok minggu bersetub...
2923,KD-43406,ingin menambah tinggi badan,ingin menambah tinggi badanselamat malam dok ...
2924,KD-43411,anak 1 tahun boleh dikasih nasi ?,anak 1 tahun boleh dikasih nasi ?bayi berumur ...
2925,KD-43446,mencegah infeksi virus zika,mencegah infeksi virus zikaselamat siang dok b...


In [18]:
#Word Embedding using Word2Vec

# Class untuk enkapsulasi beberapa fungsionalitas yang ada

from gensim import models

class Word2VecModel:
    #Yoga
    # pretrained_file = "/content/drive/MyDrive/Colab Notebooks/Data Mining/Data TK 1 Text Mining/word2vec.id.300d.txt.gz"
    #Amanda
    pretrained_file = "/content/drive/MyDrive/Data TK 1 Text Mining/word2vec.id.300d.txt.gz"
    vector_size = 300
    model = models.KeyedVectors.load_word2vec_format(pretrained_file)

    # Fungsi untuk mencari representasi vektor dari suatu kata  
    def get_word_embedding(self, word):
        try:
            return self.model[word]
        except: # Melakukan handling terhadap OOV
            return np.nan
    
    # Fungsi untuk mencari representasi vektor dari suatu kalimat
    def get_sentence_embedding(self, sent):
        lst_token = re.findall("[^\s]{3,}", sent.lower())
        rep = []
        for token in lst_token:
            try:
                token_rep = self.model[token]
                rep.append(token_rep)
            except: # Melakukan handling terhadap OOV
                continue
        return np.mean(rep, axis=0)
    
    # Fungsi untuk mengubah setiap teks pada suatu corpus menjadi representasi vektornya
    def transform(self, corpus):
        res = []
        for doc in corpus:
            tmp = self.get_sentence_embedding(doc)
            if type(tmp) == np.float64: # Melakukan handling terhadap OOV
                tmp = np.zeros(self.vector_size)
            res.append(tmp)
        return np.array(res)

In [19]:
model_word2vec = Word2VecModel()

In [20]:
X_train = training_data['isi']
X_test = testing_data['isi']

X_train_w2v = model_word2vec.transform(X_train)
X_test_w2v = model_word2vec.transform(X_test)

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [21]:
y_train = training_data.copy()
y_train.drop(columns=['id','judul','isi'], inplace=True)

In [23]:
KebidananArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Kebidanan dan Kandungan" or row['label_1'] == "Kebidanan dan Kandungan" or row['label_2'] == "Kebidanan dan Kandungan"or row['label_3'] == "Kebidanan dan Kandungan"or row['label_4'] == "Kebidanan dan Kandungan"or row['label_5'] == "Kebidanan dan Kandungan"or row['label_6'] == "Kebidanan dan Kandungan"or row['label_7'] == "Kebidanan dan Kandungan"or row['label_8'] == "Kebidanan dan Kandungan":
    KebidananArray.append("1")
  else:
    KebidananArray.append("0")

DalamArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Penyakit Dalam" or row['label_1'] == "Penyakit Dalam" or row['label_2'] == "Penyakit Dalam" or row['label_3'] == "Penyakit Dalam" or row['label_4'] == "Penyakit Dalam" or row['label_5'] == "Penyakit Dalam" or row['label_6'] == "Penyakit Dalam" or row['label_7'] == "Penyakit Dalam" or row['label_8'] == "Penyakit Dalam":
    DalamArray.append("1")
  else:
    DalamArray.append("0")
AnakArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Kesehatan Anak" or row['label_1'] == "Kesehatan Anak" or row['label_2'] == "Kesehatan Anak" or row['label_3'] == "Kesehatan Anak" or row['label_4'] == "Kesehatan Anak" or row['label_5'] == "Kesehatan Anak" or row['label_6'] == "Kesehatan Anak" or row['label_7'] == "Kesehatan Anak" or row['label_8'] == "Kesehatan Anak":
    AnakArray.append("1")
  else:
    AnakArray.append("0")

KulitArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Kesehatan Kulit dan Kelamin" or row['label_1'] == "Kesehatan Kulit dan Kelamin" or row['label_2'] == "Kesehatan Kulit dan Kelamin" or row['label_3'] == "Kesehatan Kulit dan Kelamin" or row['label_4'] == "Kesehatan Kulit dan Kelamin" or row['label_5'] == "Kesehatan Kulit dan Kelamin"or row['label_6'] == "Kesehatan Kulit dan Kelamin" or row['label_7'] == "Kesehatan Kulit dan Kelamin" or row['label_8'] == "Kesehatan Kulit dan Kelamin":
    KulitArray.append("1")
  else:
    KulitArray.append("0")

GiziArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Kesehatan Gizi" or row['label_1'] == "Kesehatan Gizi" or row['label_2'] == "Kesehatan Gizi"or row['label_3'] == "Kesehatan Gizi" or row['label_4'] == "Kesehatan Gizi"or row['label_5'] == "Kesehatan Gizi" or row['label_6'] == "Kesehatan Gizi"or row['label_7'] == "Kesehatan Gizi" or row['label_8'] == "Kesehatan Gizi":
    GiziArray.append("1")
  else:
    GiziArray.append("0")

THTArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)" or row['label_1'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)" or row['label_2'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)"or row['label_3'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)" or row['label_4'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)"or row['label_5'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)" or row['label_6'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)"or row['label_7'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)" or row['label_8'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)":
    THTArray.append("1")
  else:
    THTArray.append("0")

GigiArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Gigi" or row['label_1'] == "Gigi" or row['label_2'] == "Gigi"or row['label_3'] == "Gigi" or row['label_4'] == "Gigi"or row['label_5'] == "Gigi" or row['label_6'] == "Gigi"or row['label_7'] == "Gigi" or row['label_8'] == "Gigi":
    GigiArray.append("1")
  else:
    GigiArray.append("0")

MataArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Kesehatan Mata" or row['label_1'] == "Kesehatan Mata" or row['label_2'] == "Kesehatan Mata"or row['label_3'] == "Kesehatan Mata" or row['label_4'] == "Kesehatan Mata"or row['label_5'] == "Kesehatan Mata" or row['label_6'] == "Kesehatan Mata"or row['label_7'] == "Kesehatan Mata" or row['label_8'] == "Kesehatan Mata":
    MataArray.append("1")
  else:
    MataArray.append("0")

BedahArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Bedah" or row['label_1'] == "Bedah" or row['label_2'] == "Bedah"or row['label_3'] == "Bedah" or row['label_4'] == "Bedah"or row['label_5'] == "Bedah" or row['label_6'] == "Bedah"or row['label_7'] == "Bedah" or row['label_8'] == "Bedah":
    BedahArray.append("1")
  else:
    BedahArray.append("0")

KesehatanJiwaArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Kesehatan Jiwa" or row['label_1'] == "Kesehatan Jiwa" or row['label_2'] == "Kesehatan Jiwa"or row['label_3'] == "Kesehatan Jiwa" or row['label_4'] == "Kesehatan Jiwa"or row['label_5'] == "Kesehatan Jiwa" or row['label_6'] == "Kesehatan Jiwa"or row['label_7'] == "Kesehatan Jiwa" or row['label_8'] == "Kesehatan Jiwa":
    KesehatanJiwaArray.append("1")
  else:
    KesehatanJiwaArray.append("0")

TulangArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Ortopedi (Tulang)" or row['label_1'] == "Ortopedi (Tulang)" or row['label_2'] == "Ortopedi (Tulang)"or row['label_3'] == "Ortopedi (Tulang)" or row['label_4'] == "Ortopedi (Tulang)"or row['label_5'] == "Ortopedi (Tulang)" or row['label_6'] == "Ortopedi (Tulang)"or row['label_7'] == "Ortopedi (Tulang)" or row['label_8'] == "Ortopedi (Tulang)":
    TulangArray.append("1")
  else:
    TulangArray.append("0")

JantungArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Jantung dan Pembuluh Darah" or row['label_1'] == "Jantung dan Pembuluh Darah" or row['label_2'] == "Jantung dan Pembuluh Darah" or row['label_3'] == "Jantung dan Pembuluh Darah" or row['label_4'] == "Jantung dan Pembuluh Darah" or row['label_5'] == "Jantung dan Pembuluh Darah" or row['label_6'] == "Jantung dan Pembuluh Darah" or row['label_7'] == "Jantung dan Pembuluh Darah" or row['label_8'] == "Jantung dan Pembuluh Darah":
    JantungArray.append("1")
  else:
    JantungArray.append("0")

UrologiArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Urologi" or row['label_1'] == "Urologi" or row['label_2'] == "Urologi"or row['label_3'] == "Urologi" or row['label_4'] == "Urologi"or row['label_5'] == "Urologi" or row['label_6'] == "Urologi"or row['label_7'] == "Urologi" or row['label_8'] == "Urologi":
    UrologiArray.append("1")
  else:
    UrologiArray.append("0")

SarafArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Saraf" or row['label_1'] == "Saraf" or row['label_2'] == "Saraf"or row['label_3'] == "Saraf" or row['label_4'] == "Saraf"or row['label_5'] == "Saraf" or row['label_6'] == "Saraf"or row['label_7'] == "Saraf" or row['label_8'] == "Saraf":
    SarafArray.append("1")
  else:
    SarafArray.append("0")

ParuArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Pulmonologi (Paru)" or row['label_1'] == "Pulmonologi (Paru)" or row['label_2'] == "Pulmonologi (Paru)"or row['label_3'] == "Pulmonologi (Paru)" or row['label_4'] == "Pulmonologi (Paru)"or row['label_5'] == "Pulmonologi (Paru)" or row['label_6'] == "Pulmonologi (Paru)"or row['label_7'] == "Pulmonologi (Paru)" or row['label_8'] == "Pulmonologi (Paru)":
    ParuArray.append("1")
  else:
    ParuArray.append("0")

In [29]:
umumArr = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Umum" or row['label_1'] == "Umum" or row['label_2'] == "Umum" or row['label_3'] == "Umum" or row['label_4'] == "Umum"or row['label_5'] == "Umum" or row['label_6'] == "Umum"or row['label_7'] == "Umum" or row['label_8'] == "Umum":
    umumArr.append("1")
  else:
    umumArr.append("0")

In [30]:
UmumArray = umumArr

In [32]:
y = [KebidananArray, DalamArray, AnakArray, KulitArray, GiziArray, THTArray, GigiArray, MataArray, BedahArray, KesehatanJiwaArray, TulangArray, JantungArray, UrologiArray, SarafArray, ParuArray, UmumArray]

y_train["KebidananArray"] = KebidananArray
y_train["DalamArray"] = DalamArray
y_train["AnakArray"] = AnakArray
y_train["KulitArray"] = KulitArray
y_train["GiziArray"] = GiziArray
y_train["THTArray"] = THTArray
y_train["GigiArray"] = GigiArray
y_train["MataArray"] = MataArray
y_train["BedahArray"] = BedahArray
y_train["KesehatanJiwaArray"] = KesehatanJiwaArray
y_train["TulangArray"] = TulangArray
y_train["JantungArray"] = JantungArray
y_train["UrologiArray"] = UrologiArray
y_train["SarafArray"] = SarafArray
y_train["ParuArray"] = ParuArray
y_train["UmumArray"] = UmumArray

## **Ensemble Learning with AdaBoost & XGBoost**

In [34]:
from sklearn.ensemble import AdaBoostClassifier

# **model bidan**

In [68]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test
X_train_ens, X_test_ens, Y_train_ens, Y_test_ens = train_test_split(X_train_w2v, y[0], test_size = 0.3, random_state = 100)
scaler = StandardScaler()
X_train_ens = scaler.fit_transform(X_train_ens)
X_test_ens = scaler.transform(X_test_ens)

In [69]:
adaboost_bidan = AdaBoostClassifier(n_estimators = 50, learning_rate = 1.0).fit(X_train_ens, Y_train_ens)

In [41]:
pred_res_bidan = adaboost_bidan.predict(X_test_ens)

# **model penyakit dalam**

In [70]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test
X_train_ens, X_test_ens, Y_train_ens, Y_test_ens = train_test_split(X_train_w2v, y[1], test_size = 0.3, random_state = 100)
scaler = StandardScaler()
X_train_ens = scaler.fit_transform(X_train_ens)
X_test_ens = scaler.transform(X_test_ens)

adaboost_dalam = AdaBoostClassifier(n_estimators = 50, learning_rate =1.0).fit(X_train_ens, Y_train_ens)

pred_res_dalam = adaboost_dalam.predict(X_test_ens)

# **model anak**

In [71]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test
X_train_ens, X_test_ens, Y_train_ens, Y_test_ens = train_test_split(X_train_w2v, y[2], test_size = 0.3, random_state = 100)
scaler = StandardScaler()
X_train_ens = scaler.fit_transform(X_train_ens)
X_test_ens = scaler.transform(X_test_ens)

adaboost_anak = AdaBoostClassifier(n_estimators = 50, learning_rate = 1.0).fit(X_train_ens, Y_train_ens)

pred_res_anak = adaboost_anak.predict(X_test_ens)

# **model kulit kelamin**

In [72]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test
X_train_ens, X_test_ens, Y_train_ens, Y_test_ens = train_test_split(X_train_w2v, y[3], test_size = 0.3, random_state = 100)
scaler = StandardScaler()
X_train_ens = scaler.fit_transform(X_train_ens)
X_test_ens = scaler.transform(X_test_ens)

adaboost_kulit = AdaBoostClassifier(n_estimators = 50, learning_rate = 1.0).fit(X_train_ens, Y_train_ens)

pred_res_kulit = adaboost_kulit.predict(X_test_ens)

# **model gizi**

In [73]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test
X_train_ens, X_test_ens, Y_train_ens, Y_test_ens = train_test_split(X_train_w2v, y[4], test_size = 0.3, random_state = 100)
scaler = StandardScaler()
X_train_ens = scaler.fit_transform(X_train_ens)
X_test_ens = scaler.transform(X_test_ens)

adaboost_gizi = AdaBoostClassifier(n_estimators = 50, learning_rate = 1.0).fit(X_train_ens, Y_train_ens)

pred_res_gizi = adaboost_gizi.predict(X_test_ens)

# **model THT**

In [74]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test
X_train_ens, X_test_ens, Y_train_ens, Y_test_ens = train_test_split(X_train_w2v, y[5], test_size = 0.3, random_state = 100)
scaler = StandardScaler()
X_train_ens = scaler.fit_transform(X_train_ens)
X_test_ens = scaler.transform(X_test_ens)

adaboost_tht = AdaBoostClassifier(n_estimators = 50, learning_rate = 1.0).fit(X_train_ens, Y_train_ens)

pred_res_tht = adaboost_tht.predict(X_test_ens)

# **model Gigi**

In [75]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test
X_train_ens, X_test_ens, Y_train_ens, Y_test_ens = train_test_split(X_train_w2v, y[6], test_size = 0.3, random_state = 100)
scaler = StandardScaler()
X_train_ens = scaler.fit_transform(X_train_ens)
X_test_ens = scaler.transform(X_test_ens)

adaboost_gigi = AdaBoostClassifier(n_estimators = 50, learning_rate = 1.0).fit(X_train_ens, Y_train_ens)

pred_res_gigi = adaboost_gigi.predict(X_test_ens)

# **model Mata**

In [76]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test
X_train_ens, X_test_ens, Y_train_ens, Y_test_ens = train_test_split(X_train_w2v, y[7], test_size = 0.3, random_state = 100)
scaler = StandardScaler()
X_train_ens = scaler.fit_transform(X_train_ens)
X_test_ens = scaler.transform(X_test_ens)

adaboost_mata = AdaBoostClassifier(n_estimators = 50, learning_rate = 0.2).fit(X_train_ens, Y_train_ens)

pred_res_mata = adaboost_mata.predict(X_test_ens)

# **model Bedah**

In [50]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test
X_train_ens, X_test_ens, Y_train_ens, Y_test_ens = train_test_split(X_train_w2v, y[8], test_size = 0.3, random_state = 100)
scaler = StandardScaler()
X_train_ens = scaler.fit_transform(X_train_ens)
X_test_ens = scaler.transform(X_test_ens)

adaboost_bedah = AdaBoostClassifier(n_estimators = 50, learning_rate = 0.2).fit(X_train_ens, Y_train_ens)

pred_res_bedah = adaboost_bedah.predict(X_test_ens)

# **model Kesehatan Jiwa**

In [77]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test
X_train_ens, X_test_ens, Y_train_ens, Y_test_ens = train_test_split(X_train_w2v, y[9], test_size = 0.3, random_state = 100)
scaler = StandardScaler()
X_train_ens = scaler.fit_transform(X_train_ens)
X_test_ens = scaler.transform(X_test_ens)

adaboost_jiwa = AdaBoostClassifier(n_estimators = 50, learning_rate = 1.0).fit(X_train_ens, Y_train_ens)

pred_res_jiwa = adaboost_jiwa.predict(X_test_ens)

# **model Tulang**

In [78]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test
X_train_ens, X_test_ens, Y_train_ens, Y_test_ens = train_test_split(X_train_w2v, y[10], test_size = 0.3, random_state = 100)
scaler = StandardScaler()
X_train_ens = scaler.fit_transform(X_train_ens)
X_test_ens = scaler.transform(X_test_ens)

adaboost_tulang = AdaBoostClassifier(n_estimators = 50, learning_rate = 1.0).fit(X_train_ens, Y_train_ens)

pred_res_tulang = adaboost_tulang.predict(X_test_ens)

# **model Jantung**

In [79]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test
X_train_ens, X_test_ens, Y_train_ens, Y_test_ens = train_test_split(X_train_w2v, y[11], test_size = 0.3, random_state = 100)
scaler = StandardScaler()
X_train_ens = scaler.fit_transform(X_train_ens)
X_test_ens = scaler.transform(X_test_ens)

adaboost_jantung = AdaBoostClassifier(n_estimators = 50, learning_rate = 1.0).fit(X_train_ens, Y_train_ens)

pred_res_jantung = adaboost_jantung.predict(X_test_ens)

# **model Urologi**

In [80]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test
X_train_ens, X_test_ens, Y_train_ens, Y_test_ens = train_test_split(X_train_w2v, y[12], test_size = 0.3, random_state = 100)
scaler = StandardScaler()
X_train_ens = scaler.fit_transform(X_train_ens)
X_test_ens = scaler.transform(X_test_ens)

adaboost_uro = AdaBoostClassifier(n_estimators = 50, learning_rate = 1.0).fit(X_train_ens, Y_train_ens)

pred_res_uro = adaboost_uro.predict(X_test_ens)

# **model Saraf**

In [81]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test
X_train_ens, X_test_ens, Y_train_ens, Y_test_ens = train_test_split(X_train_w2v, y[13], test_size = 0.3, random_state = 100)
scaler = StandardScaler()
X_train_ens = scaler.fit_transform(X_train_ens)
X_test_ens = scaler.transform(X_test_ens)

adaboost_saraf = AdaBoostClassifier(n_estimators = 50, learning_rate = 1.0).fit(X_train_ens, Y_train_ens)

pred_res_saraf = adaboost_saraf.predict(X_test_ens)

# **model Paru**

In [82]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test
X_train_ens, X_test_ens, Y_train_ens, Y_test_ens = train_test_split(X_train_w2v, y[14], test_size = 0.3, random_state = 100)
scaler = StandardScaler()
X_train_ens = scaler.fit_transform(X_train_ens)
X_test_ens = scaler.transform(X_test_ens)

adaboost_paru = AdaBoostClassifier(n_estimators = 50, learning_rate = 1.0).fit(X_train_ens, Y_train_ens)

pred_res_paru = adaboost_paru.predict(X_test_ens)

# **model Umum**

In [83]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test
X_train_ens, X_test_ens, Y_train_ens, Y_test_ens = train_test_split(X_train_w2v, y[15], test_size = 0.3, random_state = 100)
scaler = StandardScaler()
X_train_ens = scaler.fit_transform(X_train_ens)
X_test_ens = scaler.transform(X_test_ens)

adaboost_umum = AdaBoostClassifier(n_estimators = 50, learning_rate = 1.0).fit(X_train_ens, Y_train_ens)

pred_res_umum = adaboost_umum.predict(X_test_ens)

# Finalisasi

URUTAN
1. Kebidanan dan Kandungan
2. Penyakit Dalam
3. Kesehatan Anak
4. Kesehatan Kulit dan Kelamin
5. Kesehatan Gizi
6. Kesehatan Telinga, Hidung dan Tenggorokan (THT)
7. Gigi
8. Kesehatan Mata
9. Bedah
10. Kesehatan Jiwa
11. Ortopedi (Tulang)
12. Jantung dan Pembuluh Darah
13. Urologi
14. Saraf
15. Pulmonologi (Paru)
16. Umum

In [59]:
testing_duplicate = testing_data.iloc[:,[0]]

In [84]:
pred_res_bidan_testing = adaboost_bidan.predict(X_test_w2v)
len(pred_res_bidan_testing)

2927

In [85]:
pred_res_dalam_testing = adaboost_dalam.predict(X_test_w2v)
len(pred_res_dalam_testing)

2927

In [86]:
pred_res_anak_testing = adaboost_anak.predict(X_test_w2v)
len(pred_res_anak_testing)

2927

In [87]:
pred_res_kulit_testing = adaboost_kulit.predict(X_test_w2v)
pred_res_gizi_testing = adaboost_gizi.predict(X_test_w2v)
pred_res_THT_testing = adaboost_tht.predict(X_test_w2v)
pred_res_gigi_testing = adaboost_gigi.predict(X_test_w2v)
pred_res_mata_testing = adaboost_mata.predict(X_test_w2v)
pred_res_bedah_testing = adaboost_bedah.predict(X_test_w2v)
pred_res_jiwa_testing = adaboost_jiwa.predict(X_test_w2v)
pred_res_tulang_testing = adaboost_tulang.predict(X_test_w2v)
pred_res_jantung_testing = adaboost_jantung.predict(X_test_w2v)
pred_res_urologi_testing = adaboost_uro.predict(X_test_w2v)
pred_res_saraf_testing = adaboost_saraf.predict(X_test_w2v)
pred_res_paru_testing = adaboost_paru.predict(X_test_w2v)
pred_res_umum_testing = adaboost_umum.predict(X_test_w2v)

In [88]:
testing_duplicate['Kebidanan'] = pred_res_bidan_testing
testing_duplicate['Dalam'] = pred_res_dalam_testing
testing_duplicate['Anak'] = pred_res_anak_testing
testing_duplicate['Kulit'] = pred_res_kulit_testing
testing_duplicate['Gizi'] = pred_res_gizi_testing
testing_duplicate['THT'] = pred_res_THT_testing

testing_duplicate['Gigi'] = pred_res_gigi_testing
testing_duplicate['Mata'] = pred_res_mata_testing
testing_duplicate['Bedah'] = pred_res_bedah_testing
testing_duplicate['KesehatanJiwa'] = pred_res_jiwa_testing

testing_duplicate['Tulang'] = pred_res_tulang_testing
testing_duplicate['Jantung'] = pred_res_jantung_testing
testing_duplicate['Urologi'] = pred_res_urologi_testing
testing_duplicate['Saraf'] = pred_res_saraf_testing
testing_duplicate['Paru'] = pred_res_paru_testing

testing_duplicate['Umum'] = pred_res_umum_testing

In [89]:
testing_duplicate

,id,Kebidanan,Dalam,Anak,Kulit,Gizi,THT,Gigi,Mata,Bedah,KesehatanJiwa,Tulang,Jantung,Urologi,Saraf,Paru,Umum
0,DS-45,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,DS-339,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,AD-18,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,AD-66,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,AD-73,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2922,KD-43404,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2923,KD-43406,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2924,KD-43411,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1
2925,KD-43446,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [90]:
testing_duplicate.to_csv('/content/drive/MyDrive/Data TK 1 Text Mining/ensemble_adaboost1.csv', index=False, header=False)